In [1]:
suppressPackageStartupMessages({
    library(tidyverse)
    library(ggplot2)
    library(palmerpenguins)
    library(lubridate)
    library(DBI)
    library(RSQLite)
    library(httr2)
    library(rvest)
})

# SQL and Introduction to RegEx

## SQL
1. Connect to the SQL database (db for short) using `dbConnect()`
2. Make a SQL query (a question or request) using `dbGetQuery()`
3. When done, disconnect using `dbDisconnect()`
### Step 1: Connect

In [2]:
# Step 1. Make a connection to the database
conn <- dbConnect(RSQLite::SQLite(), "user_actions.db")

In [3]:
# Check the available tables
dbListTables(conn)

[1] "user_actions"

In [4]:
# Check the available variables in the table
dbListFields(conn, "user_actions")

[1] "user_id"   "username"  "email"     "action"    "timestamp"

### Step 2: SQL Query

In [5]:
# Query: Retrieve the whole table
table <- dbGetQuery(conn, "
    SELECT *
    FROM user_actions;
")
glimpse(table)

Rows: 8,040
Columns: 5
$ user_id   <int> 34, 28, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 33, 27, 27, ~
$ username  <chr> "user34", "user28", "user27", "user27", "user27", "user27", ~
$ email     <chr> "user34@email.com", "user28@email.com", "user27@email.com", ~
$ action    <chr> "signup", "signup", "login", "login", "reset_password", "log~
$ timestamp <chr> "2015-02-04 14:38:47", "2015-03-09 11:55:33", "2015-04-17 14~


In [6]:
# Query: Use `WHERE` to filter all users who has made the action signup
signup_users_log <- dbGetQuery(conn, "
    SELECT username, action, timestamp
    FROM user_actions
    WHERE action = 'signup';
")
head(signup_users_log)

,username,action,timestamp
,<chr>,<chr>,<chr>
1,user34,signup,2015-02-04 14:38:47
2,user28,signup,2015-03-09 11:55:33
3,user1,signup,2015-11-06 08:07:13
4,user24,signup,2015-12-11 15:25:07
5,user15,signup,2016-04-03 16:12:11
6,user20,signup,2016-05-28 16:40:00


In [7]:
# Query: Find the total number of log entries for each user using GROUP BY and the function COUNT(*)
log_counts <- dbGetQuery(conn, "
    SELECT user_id, username, COUNT(*) AS log_counts
    FROM user_actions
    GROUP BY user_id, username;
")
head(log_counts)

,user_id,username,log_counts
,<int>,<chr>,<int>
1,1,user1,104
2,2,user2,149
3,3,user3,108
4,4,user4,436
5,5,user5,192
6,6,user6,457


In [8]:
dbDisconnect(conn)

## RegEx
- Use `gregexpr(pattern, data)` to find all positions that matches the pattern
- Use `regexpr()` to find the first position that matches the pattern
- Use `regmatches()` to extract the substring(s)
### Example: Extract the substrings starting with Hashtag

In [9]:
# Read the txt file
comments <- readLines("comments.txt")

# Select a random comment to search
comment <- comments[37]
comment

# Define the pattern
pattern <- "#\\w+"

# Find the first position that matches the pattern
match <- regexpr(pattern, comment)

# Find all positions that matches the pattern
matches <- gregexpr(pattern, comment)

# Extract the substring(s)
regmatches(comment, match)
regmatches(comment, matches)

[1] "37. \"Coding challenges are rewarding. #coding #python\""

[1] "#coding"

[[1]]
[1] "#coding" "#python"